In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pulp
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df11 = pd.read_excel('../excels/1.xlsx', sheet_name='乡村的现有耕地')
# print(df11)
df12 = pd.read_excel('../excels/1.xlsx', sheet_name='乡村种植的农作物')
df12.fillna(method='ffill', inplace=True)
# print(df12)
df21 = pd.read_excel('../excels/2.xlsx', sheet_name='2023年的农作物种植情况')
df21.fillna(method='ffill', inplace=True)
# print(df21)
df22 = pd.read_excel('../excels/2.xlsx', sheet_name='2023年统计的相关数据')
# 计算销售单价平均值
def calculate_average_price(price_str):
    prices = list(map(float, price_str.split('-')))
    return sum(prices) / len(prices)
df22['销售单价平均值/(元/斤)'] = df22['销售单价/(元/斤)'].apply(calculate_average_price)
# print(df22)
df2023_result = pd.read_excel('../excels/附件3/2023_result.xlsx')
# # 去除列名末尾的空格。很奇怪
# df2023_result.columns = [column.rstrip(' ') for column in df2023_result.columns]
# print(df2023_result.columns)

In [2]:
result1 = df21.groupby(['作物编号', '地块类型', '种植季次'])['种植面积/亩'].sum().reset_index()
result1.columns = ['作物编号', '地块类型', '种植季次', '种植面积/亩']
merged_df = pd.merge(result1, df22[['作物编号', '作物名称','作物类型' , '地块类型', '种植季次', '亩产量/斤', '种植成本/(元/亩)', '销售单价/(元/斤)']], on=['作物编号', '种植季次', '地块类型'], how='left')
merged_df['单地块类型总产量/斤'] = merged_df['种植面积/亩'] * merged_df['亩产量/斤']
merged_df['销售单价平均值'] = merged_df['销售单价/(元/斤)'].apply(calculate_average_price)
merged_df['利润'] = (merged_df['销售单价平均值'] * merged_df['单地块类型总产量/斤']) - (merged_df['种植成本/(元/亩)'] * merged_df['种植面积/亩'])
merged_df['每亩利润'] = merged_df['利润'] / merged_df['种植面积/亩']
# print(merged_df)
total_profit_2023 = merged_df.groupby(['作物编号', '作物名称'])[['利润', '每亩利润']].sum().reset_index()
total_profit_2023.columns = ['作物编号', '作物名称', '总利润/元', '每亩总利润/元']
# print(total_profit)
total_yield_2023 = merged_df.groupby(['作物编号', '作物名称'])['单地块类型总产量/斤'].sum().reset_index()
total_yield_2023.columns = ['作物编号', '作物名称', '总产量/斤']
# print(total_yield_2023)

In [3]:
class Field:
    def __init__(self, field_name, field_type, field_area, crop_and_season, season, seasonmonth, planted_crop):
        self.field_name = field_name
        self.field_type = field_type
        self.field_area = field_area
        self.crop_and_season = crop_and_season
        self.season = season
        self.seasonmonth = seasonmonth
        self.planted_crop = planted_crop

    def __str__(self):
        return f'Field(planted_crop={self.planted_crop}, field_name={self.field_name}, field_type={self.field_type}, field_area={self.field_area}, crop_and_season={self.crop_and_season}, season={self.season}, seasonmonth={self.seasonmonth})'

fields = []
for index, row in df11.iterrows():
    field_name = row['地块名称']
    field_type = row['地块类型']

    if field_type[-1] == ' ':
        # 很奇怪，普通大棚后面有个空格
        field_type = field_type.rstrip(' ')
        # print(repr(field_type))

    field_area = row['地块面积/亩']
    planted_crop = []
    # 处理种植作物和季节字符串
    crop_and_season_str = row['种植作物和季节'].rstrip('0')
    crop_and_season_groups = crop_and_season_str.split(',')
    crop_and_season = [group.split() for group in crop_and_season_groups]    
    # 初始化 seasonmonth 变量
    seasonmonth = []
    season = 0
    seasonmonth_str = row['季节的月份']
    if seasonmonth_str != 0:
        seasonmonth_groups = seasonmonth_str.split(',')
        seasonmonth = [group.split() for group in seasonmonth_groups]
    field = Field(field_name, field_type, field_area, crop_and_season, season, seasonmonth, planted_crop)
    fields.append(field)
# for field in fields:
#     print(field)
# 遍历 fields 列表并修改 season 数据
new_fields = []
for field in fields:
    if field.field_type in ['平旱地', '梯田', '山坡地']:
        field.season = '单季'
        new_fields.append(field)
    elif field.field_type == '水浇地':
        field_single = Field(field.field_name, field.field_type, field.field_area, field.crop_and_season, '单季', field.seasonmonth, field.planted_crop)
        field_first = Field(field.field_name, field.field_type, field.field_area, field.crop_and_season, '第一季', field.seasonmonth, field.planted_crop)
        field_second = Field(field.field_name, field.field_type, field.field_area, field.crop_and_season, '第二季', field.seasonmonth, field.planted_crop)
        new_fields.extend([field_single, field_first, field_second])
    elif field.field_type in ['普通大棚', '智慧大棚']:
        field_first = Field(field.field_name, field.field_type, field.field_area, field.crop_and_season, '第一季', field.seasonmonth, field.planted_crop)
        field_second = Field(field.field_name, field.field_type, field.field_area, field.crop_and_season, '第二季', field.seasonmonth, field.planted_crop)
        new_fields.extend([field_first, field_second])
for field in new_fields:
    print(field)

Field(planted_crop=[], field_name=A1, field_type=平旱地, field_area=80.0, crop_and_season=[['粮食', '单季'], ['粮食（豆类）', '单季']], season=单季, seasonmonth=[])
Field(planted_crop=[], field_name=A2, field_type=平旱地, field_area=55.0, crop_and_season=[['粮食', '单季'], ['粮食（豆类）', '单季']], season=单季, seasonmonth=[])
Field(planted_crop=[], field_name=A3, field_type=平旱地, field_area=35.0, crop_and_season=[['粮食', '单季'], ['粮食（豆类）', '单季']], season=单季, seasonmonth=[])
Field(planted_crop=[], field_name=A4, field_type=平旱地, field_area=72.0, crop_and_season=[['粮食', '单季'], ['粮食（豆类）', '单季']], season=单季, seasonmonth=[])
Field(planted_crop=[], field_name=A5, field_type=平旱地, field_area=68.0, crop_and_season=[['粮食', '单季'], ['粮食（豆类）', '单季']], season=单季, seasonmonth=[])
Field(planted_crop=[], field_name=A6, field_type=平旱地, field_area=55.0, crop_and_season=[['粮食', '单季'], ['粮食（豆类）', '单季']], season=单季, seasonmonth=[])
Field(planted_crop=[], field_name=B1, field_type=梯田, field_area=60.0, crop_and_season=[['粮食', '单季'], ['粮食（豆类）', 

In [4]:
class Crop:
    def __init__(self, crop_id, crop_name, crop_type, planting_fields, crop_price):
        self.crop_id = crop_id
        self.crop_name = crop_name
        self.crop_type = crop_type
        self.planting_fields = planting_fields
        self.crop_price = crop_price

    def __repr__(self):
        return f"Crop(crop_id={self.crop_id}, crop_name={self.crop_name}, crop_type={self.crop_type}, planting_fields={self.planting_fields}, crop_price={self.crop_price})"
crop_price_map = df22.set_index('作物编号')['销售单价平均值/(元/斤)'].to_dict()
crops = []
for index, row in df12.iterrows():
    crop_id = row['作物编号']
    crop_name = row['作物名称']
    # if crop_name[-1] == ' ':
    #     # 很奇怪，菠菜和生菜后面也有个空格
    #     crop_name = crop_name.rstrip(' ')
    #     # print(repr(crop_name))
    crop_type = row['作物类型']
    crop_price = crop_price_map.get(crop_id, None)  # 从映射中获取销售单价平均值
    # 处理种植耕地字符串
    planting_fields_str = row['种植耕地'].rstrip('0')
    planting_fields_groups = planting_fields_str.split(',')
    planting_fields = [group.split() for group in planting_fields_groups]
    # 初始化 Crop 对象并添加到列表中
    crop = Crop(crop_id, crop_name, crop_type, planting_fields, crop_price)
    crops.append(crop)

yield_map = df22.set_index(['作物编号', '地块类型', '种植季次'])['亩产量/斤'].to_dict()
# print(yield_map)
cost_map = df22.set_index(['作物编号', '地块类型', '种植季次'])['种植成本/(元/亩)'].to_dict()
# print(cost_map)
# 遍历 crops 列表，进一步遍历 planting_fields 的每一个二项列表值，追加亩产量和种植成本
for crop in crops:
    for field in crop.planting_fields:
        field_type = field[0]
        season = field[1]
        key = (crop.crop_id, field_type, season)
        
        crop_yield = yield_map.get(key, None)
        crop_cost = cost_map.get(key, None)
        
        # 当且仅当作物编号、地块类型和种植季节都相同时，追加数据
        if crop_yield is not None and crop_cost is not None:
            field.append(crop_yield)
            field.append(crop_cost)

for crop in crops:
    print(crop)    

Crop(crop_id=1, crop_name=黄豆, crop_type=粮食（豆类）, planting_fields=[['平旱地', '单季', 400, 400], ['梯田', '单季', 380, 400], ['山坡地', '单季', 360, 400]], crop_price=3.25)
Crop(crop_id=2, crop_name=黑豆, crop_type=粮食（豆类）, planting_fields=[['平旱地', '单季', 500, 400], ['梯田', '单季', 475, 400], ['山坡地', '单季', 450, 400]], crop_price=7.5)
Crop(crop_id=3, crop_name=红豆, crop_type=粮食（豆类）, planting_fields=[['平旱地', '单季', 400, 350], ['梯田', '单季', 380, 350], ['山坡地', '单季', 360, 350]], crop_price=8.25)
Crop(crop_id=4, crop_name=绿豆, crop_type=粮食（豆类）, planting_fields=[['平旱地', '单季', 350, 350], ['梯田', '单季', 330, 350], ['山坡地', '单季', 315, 350]], crop_price=7.0)
Crop(crop_id=5, crop_name=爬豆, crop_type=粮食（豆类）, planting_fields=[['平旱地', '单季', 415, 350], ['梯田', '单季', 395, 350], ['山坡地', '单季', 375, 350]], crop_price=6.75)
Crop(crop_id=6, crop_name=小麦, crop_type=粮食, planting_fields=[['平旱地', '单季', 800, 450], ['梯田', '单季', 760, 450], ['山坡地', '单季', 720, 450]], crop_price=3.5)
Crop(crop_id=7, crop_name=玉米, crop_type=粮食, planting_fields=[['平旱

决策变量：种植面积

total_yield_2023是2023年总产量

In [5]:
def get_yield(field_name, crop_name):
    # 在 new_fields 中找到对应的 Field 对象
    field = next((f for f in new_fields if f.field_name == field_name), None)
    if not field:
        return 0  # 如果未找到对应的 Field 对象，返回 0

    # 获取 field_type 和 season
    field_type = field.field_type
    season = field.season

    # 在 crops 中找到对应的 Crop 对象
    crop = next((c for c in crops if c.crop_name == crop_name), None)
    if not crop:
        return 0  # 如果未找到对应的 Crop 对象，返回 0

    # 在 crop 的 planting_fields 属性中进行匹配
    for pf in crop.planting_fields:
        if pf[0] == field_type and pf[1] == season:
            return pf[2]  # 返回第三个元素的值，即亩产量

    return 0  # 如果未找到合适的项，返回 0

def get_cost(field_name, crop_name):
    # 在 new_fields 中找到对应的 Field 对象
    field = next((f for f in new_fields if f.field_name == field_name), None)
    if not field:
        return 0  # 如果未找到对应的 Field 对象，返回 0

    # 获取 field_type 和 season
    field_type = field.field_type
    season = field.season

    # 在 crops 中找到对应的 Crop 对象
    crop = next((c for c in crops if c.crop_name == crop_name), None)
    if not crop:
        return 0  # 如果未找到对应的 Crop 对象，返回 0

    # 在 crop 的 planting_fields 属性中进行匹配
    for pf in crop.planting_fields:
        if pf[0] == field_type and pf[1] == season:
            return pf[3]  # 返回第四个元素的值，即种植成本

    return 0  # 如果未找到合适的项，返回 0

def get_total_yield_2023(crop_name, total_yield_2023):
    # 在 total_yield_2023 中找到对应的行
    row = total_yield_2023[total_yield_2023['作物名称'] == crop_name]
    
    if not row.empty:
        # 返回总产量/斤列的值
        return row['总产量/斤'].values[0]
    
    return 0  # 如果未找到对应的作物，返回 0

def check_crop_constraints(field, crop):
    for pf in crop.planting_fields:
        if pf[0] == field.field_type and pf[1] == field.season:
            return True
    return False

def check_rotation_constraints(field, crop, df_last_result):
    # 获取地块名称、作物名称和季次
    field_name = field.field_name
    crop_name = crop.crop_name
    season = field.season
    if season == '单季':
        season = '第一季'

    # 检查 df2023_result 中对应地块、季次和作物的种植面积
    for index, row in df_last_result.iterrows():
        if row['种植地块'] == field_name and row['种植季次'] == season and row[crop_name] > 0:
            return False  # 如果种植面积不为0，返回 False

    return True  # 如果种植面积为0，返回 True

# 示例调用
field_name = "A2"
crop_name = "玉米"
yield_value = get_yield(field_name, crop_name)
cost_value = get_cost(field_name, crop_name)
print(f"Yield for {crop_name} in {field_name}: {yield_value}")
print(f"Cost for {crop_name} in {field_name}: {cost_value}")
total_yield = get_total_yield_2023(crop_name, total_yield_2023)
print(f"Total yield for {crop_name} in 2023: {total_yield}")
print(check_crop_constraints(new_fields[0], crops[0])) # 黄豆适合在平旱地上单季种植
print(check_crop_constraints(new_fields[-9], crops[-4])) # 榆黄菇适合在普通大棚上第二季种植
print(check_rotation_constraints(new_fields[0], crops[0], df2023_result)) # 黄豆2023年第一季在A1地块未种植
print(check_rotation_constraints(new_fields[0], crops[5], df2023_result)) # 小麦2023年第一季在A1地块种植

Yield for 玉米 in A2: 1000
Cost for 玉米 in A2: 500
Total yield for 玉米 in 2023: 132750.0
True
True
True
False


In [6]:
# 创建线性规划问题
prob = pulp.LpProblem("Crop_Planting_Optimization", pulp.LpMaximize)

# 创建决策变量
variables = {}
for field in new_fields:
    for crop in crops:
        var_name = f"{field.field_name}_{crop.crop_name}_{field.season}"
        variables[(field.field_name, crop.crop_name, field.season)] = pulp.LpVariable(var_name, lowBound=0)

# 创建实际销售量变量
actual_sales = {}
for crop in crops:
    actual_sales[crop.crop_name] = pulp.LpVariable(f"actual_sales_{crop.crop_name}", lowBound=0)

# 创建超额部分变量
excess_yield = {}
for crop in crops:
    excess_yield[crop.crop_name] = pulp.LpVariable(f"excess_yield_{crop.crop_name}", lowBound=0)

In [7]:
# 输出每个变量的键值对
for varkey, varval in variables.items():
    print(varkey, varval)

('A1', '黄豆', '单季') A1_黄豆_单季
('A1', '黑豆', '单季') A1_黑豆_单季
('A1', '红豆', '单季') A1_红豆_单季
('A1', '绿豆', '单季') A1_绿豆_单季
('A1', '爬豆', '单季') A1_爬豆_单季
('A1', '小麦', '单季') A1_小麦_单季
('A1', '玉米', '单季') A1_玉米_单季
('A1', '谷子', '单季') A1_谷子_单季
('A1', '高粱', '单季') A1_高粱_单季
('A1', '黍子', '单季') A1_黍子_单季
('A1', '荞麦', '单季') A1_荞麦_单季
('A1', '南瓜', '单季') A1_南瓜_单季
('A1', '红薯', '单季') A1_红薯_单季
('A1', '莜麦', '单季') A1_莜麦_单季
('A1', '大麦', '单季') A1_大麦_单季
('A1', '水稻', '单季') A1_水稻_单季
('A1', '豇豆', '单季') A1_豇豆_单季
('A1', '刀豆', '单季') A1_刀豆_单季
('A1', '芸豆', '单季') A1_芸豆_单季
('A1', '土豆', '单季') A1_土豆_单季
('A1', '西红柿', '单季') A1_西红柿_单季
('A1', '茄子', '单季') A1_茄子_单季
('A1', '菠菜 ', '单季') A1_菠菜__单季
('A1', '青椒', '单季') A1_青椒_单季
('A1', '菜花', '单季') A1_菜花_单季
('A1', '包菜', '单季') A1_包菜_单季
('A1', '油麦菜', '单季') A1_油麦菜_单季
('A1', '小青菜', '单季') A1_小青菜_单季
('A1', '黄瓜', '单季') A1_黄瓜_单季
('A1', '生菜 ', '单季') A1_生菜__单季
('A1', '辣椒', '单季') A1_辣椒_单季
('A1', '空心菜', '单季') A1_空心菜_单季
('A1', '黄心菜', '单季') A1_黄心菜_单季
('A1', '芹菜', '单季') A1_芹菜_单季
('A1', '大白菜', '单季') A1_大白菜_单季
('A1

In [8]:
# 定义目标函数的各个部分

# 收入部分：实际销售量 * 作物价格
revenue = pulp.lpSum([
    actual_sales[crop.crop_name] * crop.crop_price
    for crop in crops
])

# 成本部分：种植成本
cost = pulp.lpSum([
    variables[(field.field_name, crop.crop_name, field.season)] * get_cost(field, crop)
    for field in new_fields for crop in crops
])

# 超额部分处理：超额部分 * 作物价格 * 系数
k = 0.5  # 系数
excess_handling = pulp.lpSum([
    excess_yield[crop.crop_name] * crop.crop_price * k
    for crop in crops
])

# 总利润 = 收入 - 成本 + 超额部分处理
profit = revenue - cost + excess_handling

# 将目标函数添加到问题中
prob += profit

In [9]:
# 添加约束条件
# 地块面积限制
for field in new_fields:
    prob += pulp.lpSum([variables[(field.field_name, crop.crop_name, field.season)] for crop in crops]) <= field.field_area

# 作物种植限制
for field in new_fields:
    for crop in crops:
        if not check_crop_constraints(field, crop):
            prob += variables[(field.field_name, crop.crop_name, field.season)] == 0

# 重茬限制
for field in new_fields:
    for crop in crops:
        if not check_rotation_constraints(field, crop, df2023_result):
            prob += variables[(field.field_name, crop.crop_name, field.season)] == 0

# 豆类作物种植要求
# for field in new_fields:
#     prob += pulp.lpSum([variables[(field.field_name, crop.crop_name, field.season)] 
#                         for crop in crops if crop.crop_type in ['粮食（豆类）', '蔬菜（豆类）']]) >= 1

# 实际销售量约束
for crop in crops:
    actual_yield = pulp.lpSum([
        variables[(field.field_name, crop.crop_name, field.season)] * get_yield(field.field_name, crop.crop_name)
        for field in new_fields
    ])
    expected_yield = get_total_yield_2023(crop.crop_name, total_yield_2023)
    prob += actual_sales[crop.crop_name] <= actual_yield
    prob += actual_sales[crop.crop_name] <= expected_yield

    # 超额部分约束
    prob += excess_yield[crop.crop_name] >= actual_yield - expected_yield
    prob += excess_yield[crop.crop_name] >= 0

In [10]:
# 求解问题
prob.solve()

-2

In [11]:
print("Status:", pulp.LpStatus[prob.status])
print("Objective:", pulp.value(prob.objective))

# # 打印所有变量及其值
# for v in prob.variables():
#     print(v.name, "=", v.varValue)

# 打印所有约束条件
for name, constraint in prob.constraints.items():
    print(f"{name}: {constraint}")

# # 打印模型的详细信息
# print(prob)

Status: Unbounded
Objective: 0.0
_C1: A1_刀豆_单季 + A1_包菜_单季 + A1_南瓜_单季 + A1_土豆_单季 + A1_大白菜_单季 + A1_大麦_单季 + A1_小青菜_单季 + A1_小麦_单季 + A1_榆黄菇_单季 + A1_水稻_单季 + A1_油麦菜_单季 + A1_爬豆_单季 + A1_玉米_单季 + A1_生菜__单季 + A1_白灵菇_单季 + A1_白萝卜_单季 + A1_空心菜_单季 + A1_红萝卜_单季 + A1_红薯_单季 + A1_红豆_单季 + A1_绿豆_单季 + A1_羊肚菌_单季 + A1_芸豆_单季 + A1_芹菜_单季 + A1_茄子_单季 + A1_荞麦_单季 + A1_莜麦_单季 + A1_菜花_单季 + A1_菠菜__单季 + A1_西红柿_单季 + A1_谷子_单季 + A1_豇豆_单季 + A1_辣椒_单季 + A1_青椒_单季 + A1_香菇_单季 + A1_高粱_单季 + A1_黄心菜_单季 + A1_黄瓜_单季 + A1_黄豆_单季 + A1_黍子_单季 + A1_黑豆_单季 <= 80.0
_C2: A2_刀豆_单季 + A2_包菜_单季 + A2_南瓜_单季 + A2_土豆_单季 + A2_大白菜_单季 + A2_大麦_单季 + A2_小青菜_单季 + A2_小麦_单季 + A2_榆黄菇_单季 + A2_水稻_单季 + A2_油麦菜_单季 + A2_爬豆_单季 + A2_玉米_单季 + A2_生菜__单季 + A2_白灵菇_单季 + A2_白萝卜_单季 + A2_空心菜_单季 + A2_红萝卜_单季 + A2_红薯_单季 + A2_红豆_单季 + A2_绿豆_单季 + A2_羊肚菌_单季 + A2_芸豆_单季 + A2_芹菜_单季 + A2_茄子_单季 + A2_荞麦_单季 + A2_莜麦_单季 + A2_菜花_单季 + A2_菠菜__单季 + A2_西红柿_单季 + A2_谷子_单季 + A2_豇豆_单季 + A2_辣椒_单季 + A2_青椒_单季 + A2_香菇_单季 + A2_高粱_单季 + A2_黄心菜_单季 + A2_黄瓜_单季 + A2_黄豆_单季 + A2_黍子_单季 + A2_黑豆_单季 <= 55.0
_C3: A3_刀豆_单季 + A

In [12]:
print("Status:", pulp.LpStatus[prob.status])
for v in prob.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue)

Status: Unbounded


In [13]:
# 更新 new_fields 列表中的种植情况
for v in variables.values():
    # 解析 v.name 得到 field_name, crop_name 和 season
    print(v.name.split('_'))

#     # 找到对应的 Field 对象
#     for field in new_fields:
#         if field.field_name == field_name and field.season == season:
#             # 向 planted_crop 元素追加一个二元列表
#             field.planted_crop.append([crop_name, v.varValue])
#             break

# # 打印结果
# for field in new_fields:
#     print(field)

['A1', '黄豆', '单季']
['A1', '黑豆', '单季']
['A1', '红豆', '单季']
['A1', '绿豆', '单季']
['A1', '爬豆', '单季']
['A1', '小麦', '单季']
['A1', '玉米', '单季']
['A1', '谷子', '单季']
['A1', '高粱', '单季']
['A1', '黍子', '单季']
['A1', '荞麦', '单季']
['A1', '南瓜', '单季']
['A1', '红薯', '单季']
['A1', '莜麦', '单季']
['A1', '大麦', '单季']
['A1', '水稻', '单季']
['A1', '豇豆', '单季']
['A1', '刀豆', '单季']
['A1', '芸豆', '单季']
['A1', '土豆', '单季']
['A1', '西红柿', '单季']
['A1', '茄子', '单季']
['A1', '菠菜', '', '单季']
['A1', '青椒', '单季']
['A1', '菜花', '单季']
['A1', '包菜', '单季']
['A1', '油麦菜', '单季']
['A1', '小青菜', '单季']
['A1', '黄瓜', '单季']
['A1', '生菜', '', '单季']
['A1', '辣椒', '单季']
['A1', '空心菜', '单季']
['A1', '黄心菜', '单季']
['A1', '芹菜', '单季']
['A1', '大白菜', '单季']
['A1', '白萝卜', '单季']
['A1', '红萝卜', '单季']
['A1', '榆黄菇', '单季']
['A1', '香菇', '单季']
['A1', '白灵菇', '单季']
['A1', '羊肚菌', '单季']
['A2', '黄豆', '单季']
['A2', '黑豆', '单季']
['A2', '红豆', '单季']
['A2', '绿豆', '单季']
['A2', '爬豆', '单季']
['A2', '小麦', '单季']
['A2', '玉米', '单季']
['A2', '谷子', '单季']
['A2', '高粱', '单季']
['A2', '黍子', '单季']
['A2', '荞麦',